## Levantar la data

In [ ]:
import pandas as pd
import numpy as np
import datetime
import locale
import unicodedata

locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')

ruta = "//marte/SSEstadisticaCriminal/44_Proyecto Base Unificada 2024/Data Fuerzas/"
#"C:/Users/Usuario/Desktop/Data Fuerzas/"
mes = "noviembre"
fuerza = "PSA"
formato = ".xlsx"

semana = "_4"

# OJO PSA SIEMPRE MANDA EN LOS SEMANALES TODO LA INFO, ES DECIR NO MANDA INCREMENTALES. 

año = 2024
numero_mes = 11
ultimo_dia = 27

file = ruta + mes + f"/{fuerza}_{mes}{semana}" + formato
file

In [ ]:
header = 2

procedimiento = pd.read_excel(file, header=header, sheet_name="GEOG. PROCEDIMIENTO")
controlado = pd.read_excel(file, header=header, sheet_name="VEHI. Y PERSO. CONTROLADAS")
elementos = pd.read_excel(file, header=header, sheet_name="PERSONAL Y ELEMENTOS AFECTADOS")
detenidos = pd.read_excel(file, header=header, sheet_name="DETENIDOS Y APREHENDIDOS")
incautaciones = pd.read_excel(file, header=header, sheet_name="INCAUTACIONES")
try:
    trata = pd.read_excel(file, header=header, sheet_name="TRATA O TRAFIC PERSONAS")
except ValueError:
    trata = pd.read_excel(file, header=header, sheet_name="TRATA O TRAFICO PERSONAS")
otros_eventos = pd.read_excel(file, header=header, sheet_name="OTROS EVENTOS")
otros_delitos = pd.read_excel(file, header=header, sheet_name="OTROS DELITOS")
fallecidos = pd.read_excel(file, header=header, sheet_name="FALLECIDOS")
fallecidos = fallecidos.rename(columns={"FUERZA DE SEGURIDAD": "FUERZA_INTERVINIENTE"})
abatidos = pd.read_excel(file, header=header, sheet_name="ABATIDOS")
abatidos = abatidos.rename(columns={"FUERZA DE SEGURIDAD": "FUERZA_INTERVINIENTE"})

# print("procedimiento:", procedimiento.columns, "\n")
# print("controlado:", controlado.columns, "\n")
# print("elementos:", elementos.columns, "\n")
# print("detenidos:", detenidos.columns, "\n")
# print("incautaciones:", incautaciones.columns, "\n")
# print("trata:", trata.columns, "\n")
# print("otros_eventos:", otros_eventos.columns, "\n")
# print("otros_delitos:", otros_delitos.columns, "\n")
# print("fallecidos:", fallecidos.columns, "\n")
# print("abatidos:", abatidos.columns, "\n")

In [1]:
## Fecha

In [ ]:
enteros_df = procedimiento[procedimiento['FECHA'].apply(lambda x: isinstance(x, int))]
string_df = procedimiento[procedimiento['FECHA'].apply(lambda x: isinstance(x, str))]
fechas_df = procedimiento[procedimiento['FECHA'].apply(lambda x: isinstance(x, datetime.datetime))]

if not string_df.empty:
    string_df.loc[:, 'FECHA'] = pd.to_datetime(string_df['FECHA'], format='%d/%m/%Y').dt.date

if not fechas_df.empty:
    fechas_df = fechas_df.copy()
    fechas_df['FECHA'] = pd.to_datetime(procedimiento['FECHA'], format='%d/%m/%Y').dt.date

if not enteros_df.empty:
    enteros_df = enteros_df.copy()
    enteros_df['FECHA'] = enteros_df['FECHA'].astype(int)
    enteros_df.loc[:, 'FECHA'] = pd.to_datetime(enteros_df['FECHA'], origin='1899-12-30', unit='D').dt.date

procedimiento_casted = pd.concat([fechas_df, enteros_df, string_df]).sort_values(by='FECHA')

if len(procedimiento) != len(procedimiento_casted):
    print("warning: se perdieron filas en el casting")

print(f"desde: {procedimiento_casted['FECHA'].min()}, hasta: max_date = {procedimiento_casted['FECHA'].max()}")



procedimiento_filtered = procedimiento_casted[
# Linea para cuando las semana no tienen distintos meses
(procedimiento_casted['FECHA'] >= datetime.date(año, numero_mes, 1)) & (procedimiento_casted['FECHA'] <= datetime.date(año, numero_mes, ultimo_dia))]

# Linea para cuando las semana tienen distintos meses modificar el mes en el primera poniendo el mes anterior
#(procedimiento_casted['FECHA'] >= datetime.date(año, 10, 1)) & (procedimiento_casted['FECHA'] <= datetime.date(año, numero_mes, ultimo_dia))]

fechas_fuera_de_rango = len(procedimiento) - len(procedimiento_filtered)

if fechas_fuera_de_rango != 0:
    print(f"warning: se perdieron {fechas_fuera_de_rango} filas en el filtro")

procedimiento_filtered['FECHA'].unique()

## ID y columnas

In [ ]:
df = [
    procedimiento_filtered,
    controlado,
    elementos,
    detenidos,
    incautaciones,
    trata,
    otros_eventos,
    otros_delitos,
    fallecidos,
    abatidos
]

def clean_df(df):
    df = df.copy()
    df = df.loc[:, ~df.columns.str.startswith('unnamed')]
        
    df.replace('-', np.nan, inplace=True)
    df.replace('- ', np.nan, inplace=True)
    df.replace(' -', np.nan, inplace=True)
    df.replace(' - ', np.nan, inplace=True)
    df.replace('--', np.nan, inplace=True)
    df.replace('-- ', np.nan, inplace=True)
    df.replace(' --', np.nan, inplace=True)
    df.replace(' -- ', np.nan, inplace=True)
    
    df.rename(columns=lambda x: x.lower().replace(' o ', '_').replace(' - ', '_').replace(' ', '_'), inplace=True)
    
    df[['fuerza_interviniente', 'id_operativo', 'id_procedimiento']] = df[['fuerza_interviniente', 'id_operativo', 'id_procedimiento']].astype(str)

    df['id'] = df['fuerza_interviniente'] + '_' + df['id_operativo'] + '_' + df['id_procedimiento']

    df = df[df['id'] != 'nan_nan_nan']
    
    return df

df_cleaned = [] 

for i in df:
    df_cleaned.append(clean_df(i))

keys = [
    "procedimiento",
    "controlado",
    "elementos",
    "detenidos",
    "incautaciones",
    "trata",
    "otros_eventos",
    "otros_delitos",
    "fallecidos",
    "abatidos"
]

df_dict = {}

for key, value in zip(keys, df_cleaned):
    df_dict[key] = clean_df(value)

df_dict['procedimiento']['provincia'] = df_dict['procedimiento']['provincia'].str.upper()

In [ ]:
df_vacío = []
for key, value in df_dict.items():
    if value.empty:
        df_vacío.append(key)
        
df_vacío = ', '.join(df_vacío)
df_vacío

In [ ]:
## Chequea id repetidos en envíos anteriores

id_file = ruta + "produccion/id.csv"

id_df = pd.read_csv(id_file)

if not id_df.empty:
    
    id_df = id_df[id_df.fuerza_interviniente == fuerza]
    
    # OJO PSA SIEMPRE MANDA EN LOS SEMANALES TODO LA INFO, ES DECIR NO MANDA INCREMENTALES. 
    
    id_df = id_df[id_df.mes != mes]
    valores_repetidos = df_dict['procedimiento']['id'].isin(id_df['id'])
    repetidos = df_dict['procedimiento'][valores_repetidos]
    
    
    if not repetidos.empty:
        display(repetidos[[
            "fuerza_interviniente",
            "id_operativo",
            "id_procedimiento",
            "id",
            "fecha",
        ]].sort_values(by='fecha', ascending=True))
        
    else:
        print("No hay id repetidos en envíos anteriores")

else:
    print("id_df vacío")

## Provincias

In [8]:
provincias_map = {
    "TIERRA DEL FUEGO, ANTÁRTIDA E ISLAS DEL ATLÁNTICO SUR": "TIERRA DEL FUEGO",
    "TIERRA DEL FUEGO, ANTARTIDA E ISLAS DEL ATLÁNTICO SUR": "TIERRA DEL FUEGO",
    "TIERRA DEL FUEGO, ANTÁRTIDA E ISLAS DEL ATLANTICO SUR": "TIERRA DEL FUEGO",
    "TIERRA DEL FUEGO, ANTARTIDA E ISLAS DEL ATLANTICO SUR": "TIERRA DEL FUEGO",
    "TIERRA DEL FUEGO ANTÁRTIDA E ISLAS DEL ATLÁNTICO SUR": "TIERRA DEL FUEGO",
    "TIERRA DEL FUEGO ANTARTIDA E ISLAS DEL ATLÁNTICO SUR": "TIERRA DEL FUEGO",
    "TIERRA DEL FUEGO ANTÁRTIDA E ISLAS DEL ATLANTICO SUR": "TIERRA DEL FUEGO",
    "TIERRA DEL FUEGO ANTARTIDA E ISLAS DEL ATLANTICO SUR": "TIERRA DEL FUEGO",
    "TUCUMÁN": "TUCUMAN",
    "CIUDAD AUTÓNOMA DE BUENOS AIRES": "CIUDAD AUTONOMA DE BUENOS AIRES",
    "CAPITAL FEDERAL": "CIUDAD AUTONOMA DE BUENOS AIRES",
    "CIUDAD DE BUENOS AIRES": "CIUDAD AUTONOMA DE BUENOS AIRES",
    "CÓRDOBA": "CORDOBA",
    "NEUQUÉN": "NEUQUEN",
    "ENTRE RÍOS": "ENTRE RIOS",
    "RÍO NEGRO": "RIO NEGRO",
    'DESCONOCIDA': 'SIN GEORREFERENCIAR'
}
    
df_dict['procedimiento'] = df_dict['procedimiento'].replace(provincias_map)
df_dict['fallecidos'] = df_dict['fallecidos'].replace(provincias_map)
df_dict['abatidos'] = df_dict['abatidos'].replace(provincias_map)

df_dict['procedimiento']['provincia'] = df_dict['procedimiento']['provincia'].fillna("SIN GEORREFERENCIAR")
df_dict['fallecidos']['provincia'] = df_dict['fallecidos']['provincia'].fillna("SIN GEORREFERENCIAR")
df_dict['abatidos']['provincia'] = df_dict['abatidos']['provincia'].fillna("SIN GEORREFERENCIAR")

In [9]:
df_dict['procedimiento']['zona_seguridad_fronteras'] = df_dict['procedimiento']['zona_seguridad_fronteras'].fillna('nan').astype(str)
df_dict['procedimiento']['zona_seguridad_fronteras'] = df_dict['procedimiento']['zona_seguridad_fronteras'].apply(
    lambda x: unicodedata.normalize('NFKD', x).encode('ASCII', 'ignore').decode('utf-8').upper())

df_dict['procedimiento']['departamento_partido'] = df_dict['procedimiento']['departamento_partido'].fillna('nan').astype(str)
df_dict['procedimiento']['departamento_partido'] = df_dict['procedimiento']['departamento_partido'].apply(
    lambda x: unicodedata.normalize('NFKD', x).encode('ASCII', 'ignore').decode('utf-8').upper())

df_dict['procedimiento']['localidad'] = df_dict['procedimiento']['localidad'].fillna('nan').astype(str)
df_dict['procedimiento']['localidad'] = df_dict['procedimiento']['localidad'].apply(
    lambda x: unicodedata.normalize('NFKD', x).encode('ASCII', 'ignore').decode('utf-8').upper())

In [ ]:
df_dict['procedimiento']['provincia'].unique()

## Coordenadas

In [ ]:
def corregir_coordenadas(coordenada):
    if isinstance(coordenada, float) and not np.isnan(coordenada):
        coordenada = int(coordenada)
    if isinstance(coordenada, int):
        coordenada = str(coordenada)
    if isinstance(coordenada, str) and len(coordenada) >= 4 and coordenada[0] == '-' and coordenada[3] != '.':
        coordenada = coordenada[:3] + '.' + coordenada[3:]
    if isinstance(coordenada, str) and len(coordenada) >= 4 and coordenada[0] == '-' and coordenada[3] == '.' and coordenada[4] == ',':
        coordenada = coordenada[:4] + coordenada[5:]
    return coordenada
        
df_dict['procedimiento']['latitud'] = df_dict['procedimiento']['latitud'].apply(corregir_coordenadas)
df_dict['procedimiento']['longitud'] = df_dict['procedimiento']['longitud'].apply(corregir_coordenadas)

df_dict['procedimiento']['latitud'].replace('nan', np.nan, inplace=True)
df_dict['procedimiento']['longitud'].replace('nan', np.nan, inplace=True)

In [ ]:
lat_min, lat_max = -55, -22
lon_min, lon_max = -73, -53

df_dict['procedimiento']['latitud'] = df_dict['procedimiento']['latitud'].apply(lambda x: float(x) if not pd.isnull(x) else x)
df_dict['procedimiento']['longitud'] = df_dict['procedimiento']['longitud'].apply(lambda x: float(x) if not pd.isnull(x) else x)

fuera_rango = []
for index, row in df_dict['procedimiento'].iterrows():
    if (not np.isnan(row['latitud']) and not np.isnan(row['longitud'])) and (not (lat_min <= row['latitud'] <= lat_max) or not (lon_min <= row['longitud'] <= lon_max)):
        resultado = f"Fila {index}: latitud={row['latitud']}, longitud={row['longitud']}. "
        fuera_rango.append(resultado)

fuera_rango_cnt = len(fuera_rango)
     
sin_coordenada = []
for index, row in df_dict['procedimiento'].iterrows():
    if np.isnan(row['latitud']) or np.isnan(row['longitud']):
        resultado = f"{index},"
        sin_coordenada.append(resultado)
        
sin_coordenada_cnt = len(sin_coordenada)

if fuera_rango == []:
    fuera_rango.append("ninguna. ")
    
fuera_rango = ["Fuera de rango: "] + fuera_rango
fuera_rango = ''.join(fuera_rango)

if sin_coordenada == []:
    sin_coordenada.append("ninguna.")

sin_coordenada = ["Filas sin coordenada: "] + sin_coordenada
sin_coordenada = ''.join(sin_coordenada)

coordenadas = fuera_rango + sin_coordenada
coordenadas_cnt = fuera_rango_cnt + sin_coordenada_cnt

print(coordenadas_cnt)
print(coordenadas)

## Datos faltantes

In [ ]:
print("latitud:", df_dict['procedimiento']["latitud"].isnull().sum())
print("longitud:", df_dict['procedimiento']["longitud"].isnull().sum())
sin_fecha = df_dict['procedimiento']["fecha"].isnull().sum()
print("fecha:", sin_fecha)
print("fechas fuera de rango:", fechas_fuera_de_rango)
error_fechas = sin_fecha + fechas_fuera_de_rango
print("error fechas:", error_fechas)

In [ ]:
len(df_dict['procedimiento'])

In [ ]:
for key, value in df_dict.items():
    print(f'{key}: \n')
    print(df_dict[key].isnull().sum())

## Inconsistencias de ID

In [ ]:
total = len(df_dict["procedimiento"])
total

In [ ]:
lista_chequeo = ["procedimiento", "controlado", "elementos", "otros_eventos", "fallecidos"]
duplicados = {}

for key, value in df_dict.items():
    check = df_dict[key]['id'].value_counts()
    check = check.loc[check != 1]
    if key in lista_chequeo:
        duplicados[key] = (len(check)/total) * 100
    print(key, check, "\n")

In [ ]:
procedimiento_ids = set(df_dict["procedimiento"]["id"])

faltante_id = {}
for key, df in df_dict.items():
    if key != "procedimiento":
        unique_ids = df[~df["id"].isin(procedimiento_ids)]["id"].unique()
        faltante_id[key] = unique_ids
        
print(faltante_id)

In [ ]:
casos = {}
calidad = {}

print(total)
for key, value in faltante_id.items():
    print(f'{key}:')
    print(len(value))
    casos[key] = len(value)
    try:
        porcentaje = (len(value)/total) * 100
        calidad[key] = porcentaje
        print(porcentaje)
    except ZeroDivisionError:
        porcentaje = 0.0
        print(porcentaje)
        calidad[key] = porcentaje

In [ ]:
sum(casos.values())

In [ ]:
casos

In [ ]:
sum(calidad.values())

In [ ]:
calidad

In [ ]:
duplicados

In [25]:
for key in calidad.keys():
    if key in duplicados:
        calidad[key] = calidad[key] + duplicados[key]

In [26]:
calidad["error fechas"] = (error_fechas/total) * 100
calidad["coordenadas"] = (coordenadas_cnt/total) * 100

In [ ]:
print(sum(calidad.values()))
calidad

## Resumen

In [ ]:
calidad_df = pd.DataFrame()

calidad_df['mes'] = [mes]
calidad_df['fuerza'] = [fuerza]
calidad_df['procedimientos'] = [total]
calidad_df['casos'] = [sum(casos.values())]
calidad_df['porcentaje calidad'] = [sum(calidad.values())]
calidad_df['error de fechas'] = calidad["error fechas"]
calidad_df['fechas_text'] = [f"Sin fecha: {sin_fecha}. Fechas fuera de rango: {fechas_fuera_de_rango}"]
calidad_df['error de coordenadas'] = calidad['coordenadas']
calidad_df['coordenadas_text'] = coordenadas
calidad_df['latitud nulas'] = df_dict['procedimiento']["latitud"].isnull().sum()
calidad_df['longitud nulas'] = df_dict['procedimiento']["longitud"].isnull().sum()
calidad_df['fechas nulas'] = df_dict['procedimiento']["fecha"].isnull().sum()
calidad_df['sin datos'] = df_vacío

calidad_df['casos_controlado'] = casos['controlado']
calidad_df['casos_elementos'] = casos['elementos']
calidad_df['casos_detenidos'] = "vacío" #casos['detenidos']
calidad_df['casos_incautaciones'] = casos['incautaciones']
calidad_df['casos_trata'] = casos['trata']
calidad_df['casos_otros_eventos'] = casos['otros_eventos']
calidad_df['casos_otros_delitos'] = casos['otros_delitos']

calidad_df['calidad_procedimiento'] = duplicados['procedimiento']
calidad_df['calidad_controlado'] = calidad['controlado']
calidad_df['calidad_elementos'] = calidad['elementos']
calidad_df['calidad_detenidos'] = "vacío" #calidad['detenidos']
calidad_df['calidad_incautaciones'] = calidad['incautaciones']
calidad_df['calidad_trata'] = calidad['trata']
calidad_df['calidad_otros_eventos'] = calidad['otros_eventos']
calidad_df['calidad_otros_delitos'] = calidad['otros_delitos']

calidad_df

In [29]:
calidad_df.to_csv(ruta + mes + f"/chequeo/chequeo_{fuerza}{semana}.csv", index=False)

In [30]:
for key, value in df_dict.items():
    value.to_csv(ruta + mes + f"/data controlada/{fuerza}_{mes}{semana}_{key}_controlado.csv", index=False)